In [1]:
import rospy
rospy.init_node('blockly_server', anonymous=True)
# Copied imports for setup_miro here to allow execution of other functions
from miro_msgs.msg import platform_control
from miro_constants import miro
from geometry_msgs.msg import Twist
from sensor_msgs.msg import Range, Image
from time import sleep
global q, pub, current_robot, rate
# These global variables need to be added inside of each function

In [2]:
# Colour: 230
def setup_miro():
    global q, pub, current_robot, rate
    from miro_constants import miro
    from miro_msgs.msg import platform_control
    from geometry_msgs.msg import Twist
    from time import sleep

    if ('current_robot' not in locals()):
        current_robot = 'sim01'

    pub = rospy.Publisher('/miro/' + current_robot + '/platform/control',
        platform_control, queue_size=10)
    rate = rospy.Rate(10)
    q = platform_control()

In [3]:
setup_miro()

In [4]:
# Colour: 230
def move_backward():
    global q, pub, current_robot, rate
    body_vel = Twist()
    body_vel.angular.x = 0
    body_vel.linear.x = -200
    q.body_vel = body_vel

    #ensures that at least one node is connected before sending message
    while(pub.get_num_connections() == 0):
        rate.sleep()
    pub.publish(q)
    sleep(3)	#Allow time for the move to be executed

In [5]:
move_backwards()

In [6]:
# Colour: 230
def move_forward():
    global q, pub, current_robot, rate
    body_vel = Twist()
    body_vel.angular.x = 0
    body_vel.linear.x = +200
    q.body_vel = body_vel

    #ensures that at least one node is connected before sending message
    while(pub.get_num_connections() == 0):
        rate.sleep()
    pub.publish(q)
    sleep(3)	#Allow time for the move to be executed

In [7]:
move_forwards()

In [8]:
# Colour: 230
def turn_left():
    global q, pub, current_robot, rate
    body_vel = Twist()
    body_vel.angular.z = +0.785398
    q.body_vel = body_vel

    #ensures that at least one node is connected before sending message
    while(pub.get_num_connections() == 0):
        rate.sleep()
    pub.publish(q)
    sleep(3)	#Allow time for the move to be executed

In [9]:
turn_left()

In [10]:
# Colour: 230
def turn_right():
    global q, pub, current_robot, rate
    body_vel = Twist()
    body_vel.angular.z = -0.785398
    q.body_vel = body_vel

    #ensures that at least one node is connected before sending message
    while(pub.get_num_connections() == 0):
        rate.sleep()
    pub.publish(q)
    sleep(3)	#Allow time for the move to be executed

In [11]:
turn_right()

In [12]:
# Colour: 160
def get_distance():
    global q, pub, current_robot, rate
    msg_distance = rospy.wait_for_message('/miro/' + current_robot + '/sensors/sonar_range', Range, timeout=1)
    return msg_distance.range

In [13]:
get_distance()

0.22988508641719818

In [14]:
# Interface: dropdown
# UserParameters: [Up, Down, Centre]
# CodeParameters: [L_UP, L_DOWN, L_CENTRE]
# Colour: 180
def lift_neck(dropdown_lift):
    global q, pub, current_robot, rate
    if dropdown_lift == 'L_UP':
        direction = miro.MIRO_LIFT_MIN_RAD
    elif dropdown_lift == 'L_DOWN':
        direction = miro.MIRO_LIFT_MAX_RAD
    else:
        direction = (miro.MIRO_LIFT_MIN_RAD + miro.MIRO_LIFT_MAX_RAD )/2
    q.body_vel = Twist()
    q.body_config[1] = direction
    q.body_config_speed[1] = miro.MIRO_P2U_W_LEAN_SPEED_INF

    #ensures that at least one node is connected before sending message
    while(pub.get_num_connections() == 0):
        rate.sleep()
    pub.publish(q)
    sleep(3)#Allow time for the move to be executed

In [15]:
lift_neck("L_DOWN")
lift_neck("L_UP")
lift_neck("L_CENTRE")

In [16]:
# Interface: dropdown
# UserParameters: [Up, Down, Centre]
# CodeParameters: [P_UP, P_DOWN, P_CENTRE]
# Colour: 180
def pitch_neck(dropdown_pitch):
    global q, pub, current_robot, rate
    if dropdown_pitch == 'P_UP':
        direction = miro.MIRO_PITCH_MIN_RAD
    elif dropdown_pitch == 'P_DOWN':
        direction = miro.MIRO_PITCH_MAX_RAD
    else:
        direction = (miro.MIRO_PITCH_MIN_RAD + miro.MIRO_PITCH_MAX_RAD )/2
    
    q.body_vel = Twist()
    q.body_config[3] = direction
    q.body_config_speed[3] = miro.MIRO_P2U_W_LEAN_SPEED_INF

    #ensures that at least one node is connected before sending message
    while(pub.get_num_connections() == 0):
        rate.sleep()
    pub.publish(q)
    sleep(3)	#Allow time for the move to be executed

In [17]:
pitch_neck("P_DOWN")
pitch_neck("P_UP")
pitch_neck("P_CENTRE")

In [18]:
# Interface: dropdown
# UserParameters: [Right, Left, Centre]
# CodeParameters: [Y_RIGHT, Y_LEFT, Y_CENTRE]
# Colour: 180
def yaw_neck(dropdown_yaw):
    global q, pub, current_robot, rate
    if dropdown_yaw == 'Y_RIGHT':
        direction = miro.MIRO_YAW_MAX_RAD
    elif dropdown_yaw == 'Y_LEFT':
        direction = miro.MIRO_YAW_MIN_RAD
    else:
        direction = (miro.MIRO_YAW_MIN_RAD + miro.MIRO_YAW_MAX_RAD )/2
    
    q.body_vel = Twist()
    q.body_config[2] = direction
    q.body_config_speed[2] = miro.MIRO_P2U_W_LEAN_SPEED_INF

    #ensures that at least one node is connected before sending message
    while(pub.get_num_connections() == 0):
        rate.sleep()
    pub.publish(q)
    sleep(3)	#Allow time for the move to be executed

In [19]:
yaw_neck("Y_RIGHT")
yaw_neck("Y_LEFT")
yaw_neck("Y_CENTRE")

In [21]:
get_cam_colour("0,100,0")

NameError: global name 'cam_location' is not defined

In [ ]:
setup_miro()
move_forwards()
turn_left()
move_backwards()
turn_right()
lift_neck()
yaw_neck()
pitch_neck()
